# RQ1 - Bots Functionality

### Setup Logging

In [1]:
!pip install emoji

from IPython.display import clear_output
import logging
from ipylogging import DisplayHandler, HTMLFormatter
import emoji

handler = DisplayHandler()
handler.setFormatter(HTMLFormatter())

log = logging.getLogger()
log.addHandler(handler)
log.setLevel(logging.INFO)

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/353.7 kB ? eta -:--:--
     ------------------------------------  348.2/353.7 kB 10.6 MB/s eta 0:00:01
     -------------------------------------- 353.7/353.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350833 sha256=ef42e310e0c148975611f0793bbc7631ad809b1866242fa13f2811a8b72c902f
  Stored in directory: c:\users\p308291\appdata\local\pip\cache\wheels\79\29\1c\234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Setp 0 - Modules Import, Functions, and Global Variables

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
#from minirig import load_csv_dataset, save_csv_dataset
import os
import json

data_dir = Path('../data')
bots_dataset_path = data_dir.joinpath('bots-dataset-bkp.csv')
bots_issues_dir = data_dir.joinpath('bots-issues')
#bot_dataset = load_csv_dataset(bots_dataset_path)

def plot_hist(data, x_label="", y_label="", title="", bins=5):
    # matplotlib histogram
    plt.hist(data, color = 'blue', edgecolor = 'black', bins=bins)

    # Add labels
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

def get_issues_per_bot(bot = None, n_issues = None):
    n_issues = n_issues if n_issues else 10
    issues = []
    issues_dir = Path('../data/bots-issues')
    cnt = 1
    for owner in os.listdir(issues_dir.joinpath(bot)):
        for project in os.listdir(issues_dir.joinpath(bot).joinpath(owner)):
            for issue in os.listdir(issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues')):
                issue_path = issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues').joinpath(issue)
                with open(issue_path.joinpath('json')) as f:
                    issue = json.load(f)
                    issues.append(issue['html_url'])
                    cnt += 1
                if cnt >= n_issues:
                    return issues
                    
                
    

### Step 1 - Manual Analysis of Bots Functionality

In [3]:
from ipywidgets import *

bots_dataset = pd.read_csv(bots_dataset_path)
bots_dataset.fillna(0, inplace=True)
bots_dataset['issue_count'].replace('na', 0, inplace=True)
bots_dataset.issue_count = pd.to_numeric(bots_dataset.issue_count, errors='coerce')
filtered_bots = bots_dataset.loc[bots_dataset['issue_count'] >= 100]
emojis = {1:':check_mark_button:', 0:':hollow_red_circle:'}

def handle_change (*args):
    bot_to_save = args[0]['old']
    save_dateset(bot_to_save)

    remainingW.value = f'<p>Remaining Bots: {len([i for i in filtered_bots.index if bots_dataset["completed"][i] == 0 and bots_dataset["issue_count"][i] >= 100 ])}</p>'
    bot_to_render = args[0]['new']
    
    render_bot(bot_to_render)
    
def select_issue(*args):
    text = ""
    for i in issuesW.children:
        if i.children[0].value:
            text += i.children[1].value.replace('href=', '').split(' ')[1] + '\n'
    issuesAnalyzedBkp.children[1].value = text
    print(text)
    
def save_dateset(bot):
    index = bot
    
    bots_dataset['functionality-descriptive'][index] = str(funcW.value)
    bots_dataset['is-bot'][index] = 1 if is_botT.value else 0
    bots_dataset['create'][index] = 1 if createT.value else 0
    bots_dataset['comment'][index] = 1 if commentT.value else 0
    bots_dataset['label'][index] = 1 if labelT.value else 0 
    bots_dataset['assign'][index] = 1 if assignT.value else 0 
    bots_dataset['close'][index] = 1 if closeT.value else 0
    bots_dataset['completed'][index] = 1 if completedT.value else 0
    bots_dataset['merge-pr'][index] = 1 if merge_prT.value else 0
    bots_dataset['create-milestone'][index] = 1 if milestoneT.value else 0
    bots_dataset['reopen'][index] = 1 if reopenT.value else 0
    bots_dataset['lock'][index] = 1 if lockT.value else 0
    
    bots_dataset['trigger-automated'][index] = 1 if trigger_auto.value else 0
    bots_dataset['trigger-manual'][index] = 1 if trigger_manual.value else 0
    bots_dataset['trigger-both'][index] = 1 if trigger_both.value else 0
    
    bots_dataset['issues-analyzed'][index] = str(issuesAnalyzedF.value)
    
    bots_dataset['other-remarks'][index] = str(otherRemarksF.value)
    
    bots_dataset.to_csv(bots_dataset_path, index=False)

def render_bot(bot):
    index = bot
    
    funcW.value = str(bots_dataset['functionality-descriptive'][index])

    is_botT.value = True if bots_dataset['is-bot'][index] == 1 else False

    createT.value = True if bots_dataset['create'][index] == 1 else False
    commentT.value = True if bots_dataset['comment'][index]  == 1 else False
    labelT.value = True if bots_dataset['label'][index] == 1 else False
    assignT.value = True if bots_dataset['assign'][index] == 1 else False
    closeT.value = True if bots_dataset['close'][index] == 1 else False
    merge_prT.value = True if bots_dataset['merge-pr'][index] == 1 else False
    milestoneT.value = True if bots_dataset['create-milestone'][index] == 1 else False
    reopenT.value = True if bots_dataset['create-milestone'][index] == 1 else False
    lockT.value = True if bots_dataset['lock'][index] == 1 else False

    
    completedT.value = True if bots_dataset['completed'][index] == 1 else False

    trigger_auto.value = True if bots_dataset['trigger-automated'][index] == 1 else False
    trigger_manual.value = True if bots_dataset['trigger-manual'][index] == 1 else False
    trigger_both.value = True if  bots_dataset['trigger-both'][index] == 1 else False

    otherRemarksF.value = str(bots_dataset['other-remarks'][index])
    issuesAnalyzedF.value = str(bots_dataset['issues-analyzed'][index])
    
    query_issues()

def query_issues(*args):
    bot = filtered_bots['account'][botsW.value]
    n = 10
    try:
        n = int(nIssuesW.value)
    except:
        pass
    issuesL = get_issues_per_bot(bot, n)
    
    issuesW.children = [HBox([Checkbox(description='', value=False), HTML(f'<a href={i} target="_BLANK">{i}</a>')]) for i in issuesL]
    [i.children[0].observe(select_issue, names='value') for i in issuesW.children]
       
completedT = Checkbox(description='Complete')

botsW = Dropdown(options=[(emoji.emojize(f"{filtered_bots['account'][i]} | {filtered_bots['issue_count'][i]} | {emojis[filtered_bots['completed'][i]]}"), i) for i in filtered_bots.index]) 

nIssuesW = Text(value="10")
issuesW = VBox()
remainingW = HTML(f'<p>Remaining Bots: {len([i for i in filtered_bots.index if filtered_bots["completed"][i] == 0])}</p>')

funcW = Textarea(layout=Layout(width="540px", height="150px"))

createT = Checkbox(description='Create')
commentT = Checkbox(description='Comment')
assignT = Checkbox(description='Assign')
labelT = Checkbox(description='Label')
closeT = Checkbox(description='Close')
merge_prT = Checkbox(description='Merge PR')
milestoneT = Checkbox(description='Create Milestone')
reopenT = Checkbox(description='Reopen')
lockT = Checkbox(description='Lock')
is_botT = Checkbox(description='Is a bot?')

trigger_auto = Checkbox(description='Automated')
trigger_manual = Checkbox(description='Manual')
trigger_both = Checkbox(description='Both')

otherRemarksF = Textarea(layout=Layout(width="540px", height="150px"))
issuesAnalyzedF = Textarea(layout=Layout(width="550px", height="150px"))

selectorW = HBox([VBox([HTML('Bot: '), remainingW]), botsW,is_botT]) #, #showAllT])
selectorIW = HBox([HTML('Isssues to analyze: '), nIssuesW])
functionalityW = HBox([HTML('Functionality: '), funcW])
tasksW = HBox([HTML('Tasks: '), VBox([createT, commentT, labelT,assignT, closeT, merge_prT,milestoneT,reopenT,lockT])])
triggerW = HBox([HTML('Trigger: '), VBox([trigger_auto, trigger_manual, trigger_both])])
otherRemarksW = HBox([HTML('Other Remarks: '), otherRemarksF])
issuesAnalyzedW = HBox([HTML('Issues Analyzed: '), issuesAnalyzedF])
issuesAnalyzedBkp = HBox([HTML('Issues Bkp: '), Textarea(layout=Layout(width="540px", height="150px"))])

nIssuesW.observe(query_issues)
botsW.observe(handle_change, names='value')
mainW = HBox([VBox([selectorW, tasksW, triggerW, functionalityW, issuesAnalyzedW, otherRemarksW, completedT]), VBox([HBox([HTML('Number of Issues to Analyze'),nIssuesW]),issuesAnalyzedBkp,issuesW])])
render_bot(botsW.value)
display(mainW)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)